# Modelo de Recomendación de películas

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

In [59]:
df = pd.read_csv('../clean_data/movies_model.csv')
# x = 0
# for i in range(len(df)):
#     print(i)
#     if len(df.word_bag[i]) > x:
#         x = len(df.word_bag[i])
# print(x)

In [57]:
df

,id_peli,title,word_bag
0,862,Toy Story,ToyStoryCollection Animation Comedy Family Joh...
1,8844,Jumanji,Family Adventure Fantasy JoeJohnston JoeJohnston
2,15602,Grumpier Old Men,GrumpyOldMenCollection Comedy Romance HowardDe...
3,31357,Waiting To Exhale,Comedy Romance Drama ForestWhitaker ForestWhi...
4,11862,Father Of The Bride Part Ii,Comedy CharlesShyer CharlesShyer
...,...,...,...
13131,432789,The Incredible Jessica James,Comedy Romance JimStrouse JimStrouse
13132,434873,It Stains The Sands Red,Comedy Drama Horror ColinMinihan ColinMinihan
13133,18098,Arabian Nights,Family Fantasy Drama SteveBarron SteveBarron
13134,455661,In A Heartbeat,Animation Comedy Family Romance BethDavid Est...


In [62]:
#stop_words attribute will skip all the helper words that occur english language eg: in, the , they etc.
# del(vector)
# del(cv_fvector)
# del(similarity_array)
cv_fvector = CountVectorizer(stop_words='english')
vector = cv_fvector.fit_transform(df['word_bag']).toarray()
vector.shape

(13136, 7858)

In [63]:
similarity_array = cosine_similarity(vector,vector)
similarity_array
# similarity_array = linear_kernel(vector,vector)

array([[1.        , 0.13363062, 0.13363062, ..., 0.13363062, 0.30618622,
        0.15811388],
       [0.13363062, 1.        , 0.        , ..., 0.28571429, 0.10910895,
        0.        ],
       [0.13363062, 0.        , 1.        , ..., 0.        , 0.21821789,
        0.16903085],
       ...,
       [0.13363062, 0.28571429, 0.        , ..., 1.        , 0.10910895,
        0.        ],
       [0.30618622, 0.10910895, 0.21821789, ..., 0.10910895, 1.        ,
        0.12909944],
       [0.15811388, 0.        , 0.16903085, ..., 0.        , 0.12909944,
        1.        ]])

In [64]:
sorted(list(enumerate(similarity_array[df[df.id_peli == 862].index.item()])),reverse=True,key = lambda x: x[1])[1:6]

[(1235, 0.9999999999999998),
 (998, 0.8749999999999998),
 (3693, 0.6864064729836441),
 (6178, 0.6864064729836441),
 (8071, 0.6708203932499369)]

0

In [87]:
# Recomendaciones con cosine_similarity
df = pd.read_csv('../clean_data/movies_model.csv')
# Recomendación por medio de id_peli.
recommended_five = sorted(list(enumerate(similarity_array[df[df.id_peli == 862].index.item()])),reverse=True,key = lambda x: x[1])[1:6]
# Recomendación por medio de title.
#recommended_five = sorted(list(enumerate(similarity_array[df.index[df.title.isin(['Toy Story'])].item()])),reverse=True,key = lambda x: x[1])[1:6]
lista = []
for i in recommended_five:
    lista.append(i[0])
print(f'La película que ingresaste es: {df[df.id_peli == 862].title.item()}. Las recomendaciones son: \n')
for x in df.title.loc[lista]:
    print(x)

La película que ingresaste es: Toy Story. Las recomendaciones son: 

Toy Story 2
A Bug'S Life
Cars
Cars 2
Mater And The Ghostlight


In [104]:
dfx = pd.DataFrame(columns=['id_peli','idx_recommend'])
for i in df.id_peli:
    recommended_five = sorted(list(enumerate(similarity_array[df[df.id_peli == i].index.item()])),reverse=True,key = lambda x: x[1])[1:6]
    lista = []
    for j in recommended_five:
        lista.append([i,j[0]])
    dfx = pd.concat([dfx,pd.DataFrame(lista,columns=['id_peli','idx_recommend'])])    

In [109]:
dfx = dfx.merge(df).drop('word_bag',axis=1)[['id_peli', 'title','idx_recommend']]
dfx

,id_peli,title,idx_recommend
0,862,Toy Story,1235
1,862,Toy Story,998
2,862,Toy Story,3693
3,862,Toy Story,6178
4,862,Toy Story,8071
...,...,...,...
65675,14008,Cadet Kelly,444
65676,14008,Cadet Kelly,3229
65677,14008,Cadet Kelly,3568
65678,14008,Cadet Kelly,3752


In [114]:
x = 862
lista = list(dfx[dfx.id_peli == 862].idx_recommend)
df.loc[lista][['id_peli', 'title']]

,id_peli,title
1235,863,Toy Story 2
998,9487,A Bug'S Life
3693,920,Cars
6178,49013,Cars 2
8071,13934,Mater And The Ghostlight
